# Testing convergence and order of accuracy

[AMath 586, Spring Quarter 2016](http://faculty.washington.edu/rjl/classes/am586s2016/)

This notebook illustrates testing an ODE method to determine the order of accuracy. See Appendix A and particularly A.6 for more discussion of this.

As an example, investigate the effect of errors in starting values on the accuracy of a linear multistep method.

In [ ]:
%pylab inline

### 2-stage Adams-Bashforth

Suppose we apply the 2-stage Adams-Bashforth method to the ODE $u'(t) = Au(t)$ where 
$$A = \left[\begin{array}{rr}-2&0\\3&-1\end{array}\right].$$
with initial data $u(0) = [1, ~-1]^T$.

The exact solution is $u_1(t) = e^{-2t}, ~ u_2(t) = 2e^{-t} - 3e^{-2t}$.

In [ ]:
t0 = 0.
eta = array([1., -1.])
A = array([[-2.,0.], [3.,-1.]])  # defines 2x2 matrix

# function f(u,t) (as a "lambda function")
f = lambda u,t: dot(A,u)  # matrix vector product

utrue = lambda t: array([exp(-2*t),  2*exp(-t)-3*exp(-2*t)])

In [ ]:
def AB2(tfinal,nsteps, make_plot=True, u1_method=1):
    """
    2-step Adams-Bashforth method.
    u1_method defines how U^1 is computed from U^0 = eta.
    """
    
    t = linspace(t0, tfinal, nsteps+1)
    dt = t[1] - t[0]
    
    # Array for computed solution
    # give it two rows so each column is solution at one time,
    # since function f(u) = A*u is matrix-vector multiplication
    
    U = empty((2,nsteps+1))  
    U.fill(nan)
    
    U[:,0] = eta
    
    if u1_method == 0:
        U[:,1] = eta
    elif u1_method == 1:
        U[:,1] = eta + dt * f(U[:,0], t[0])  # Forward Euler 
    else:
        print "*** Unrecognized u1_method"
        return None
    
    # Main time-stepping loop:
    for n in range(0,nsteps-1):
        U[:,n+2] = U[:,n+1] + 0.5*dt * (-f(U[:,n], t[n]) + 3*f(U[:,n+1],t[n+1]))

    if make_plot:
        tfine = linspace(t0, tfinal, 1000)  # fine grid for plotting true solution
        ufine = utrue(tfine)
        figure(figsize=(8,4))
        plot(t,U[0,:],'bo', label='u0 - AB2', markersize=5)
        plot(tfine, ufine[0,:], 'b', label='u0 - true')
        plot(t,U[1,:],'ro', label='u1 - AB2', markersize=5)
        plot(tfine, ufine[1,:], 'r', label='u1 - true')
        legend()
        title('%i steps, dt = %7.4f' % (nsteps, dt))
    
    error = U - utrue(t)
    return error

In [ ]:
error = AB2(5., 40)

### Pick a time to examine the error

It's best to pick a time before the solution has decayed close to zero or the estimates of order might not be very good (since the error also decays exponentially to zero eventually).

In [ ]:
utrue(1.)  # check the solution at t = 1

In [ ]:
tfinal = 1.
error = AB2(tfinal, 10, make_plot=False)
print "The shape of error is ",error.shape
print "Error at time %g is %s" % (tfinal, error[:,-1])
print "One-norm of error is %g" % norm(error[:,-1], 1)
print "Max-norm of error is %g" % norm(error[:,-1], inf)

In [ ]:
for nsteps in [10,20,40,80]:
    error = AB2(tfinal, nsteps, make_plot=False)
    print "nsteps = %i, 1-norm of error = %12.10f" % (nsteps, norm(error[:,-1],1))

In [ ]:
previous_E = nan
for nsteps in [10,20,40,80]:
    error = AB2(tfinal, nsteps, make_plot=False)
    E = norm(error[:,-1],1)
    if nsteps>10:
        ratio = previous_E / E
        p = log(ratio)/log(2)
        print "nsteps = %i, ratio of errors = %12.10f, estimate p = %5.2f" % (nsteps, ratio, p)
    previous_E = E

In [ ]:
nsteps_vals = [2**j for j in range(1,12)]  # "list comprehension" in Python
print "Will test with nsteps = ", nsteps_vals

In [ ]:
# Test with u1_method==1:  Euler for U^1:
E = empty(len(nsteps_vals))
for j,nsteps in enumerate(nsteps_vals):
    error = AB2(tfinal, nsteps, make_plot=False)
    E[j] = norm(error[:,-1],1)
    
loglog(nsteps_vals, E, '-o')